In [1]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms

from PIL import Image

In [2]:
BATCH_SIZE = 1
INPUT_SIZE = 64

**Prepare dataset**

In [ ]:
# ! mkdir ch2
# ! wget --quiet https://raw.githubusercontent.com/falloutdurham/beginners-pytorch-deep-learning/master/chapter2/download.py
# ! wget --quiet https://raw.githubusercontent.com/falloutdurham/beginners-pytorch-deep-learning/master/chapter2/images.csv

In [3]:
# %%capture
# % cd ch2
# !python download.py
# % cd ../

# # Takes ~ 1 hour

Remove 0 bytes images

In [4]:
files = [os.path.join(r,file) for r,d,f in os.walk("./ch2") for file in f]
files[:5]

['./ch2/images.csv',
 './ch2/download.py',
 './ch2/train/cat/1010884970_1aa4af4701.jpg',
 './ch2/train/cat/54461669_2791e8f95e.jpg',
 './ch2/train/cat/1484751958_b16fedd4c2.jpg']

In [5]:
for file in files:
    if os.path.getsize(file) == 0:
        os.remove(file)

Setting up directories pathes

In [6]:
train_path = './ch2/train/'
val_path = './ch2/val/'
test_path = './ch2/test/'

Create datasets with ImageFolder

In [7]:
img_transforms = transforms.Compose([
    transforms.Resize((INPUT_SIZE, INPUT_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], #pre-compute constants
                         std=[0.229, 0.224, 0.225])
])

In [8]:
def check_image(path):
    '''
    Function will do a sanity check to make sure PIL can actually open the file.
    Passed to the is_valid_file parameter in the ImageFolder.
    '''
    try:
        im = Image.open(path)
        return True
    except:
        return False

In [9]:
train_data = torchvision.datasets.ImageFolder(root=train_path, transform=img_transforms, is_valid_file=check_image)
val_data = torchvision.datasets.ImageFolder(root=val_path, transform=img_transforms, is_valid_file=check_image)
test_data = torchvision.datasets.ImageFolder(root=test_path, transform=img_transforms, is_valid_file=check_image)

Create data loaders

In [10]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=BATCH_SIZE)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE)

**Create model**

In [11]:
class SimpleNet(nn.Module):
    
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(INPUT_SIZE * INPUT_SIZE * 3, 84)
        self.fc2 = nn.Linear(84, 50)
        self.fc3 = nn.Linear(50, 2)
        
    def forward(self, x):
        x = x.view(-1 , INPUT_SIZE * INPUT_SIZE * 3)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x) #Final Softmax will be in the loss function - categorical crossentropy
        return x

In [12]:
simplenet = SimpleNet()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(simplenet.parameters(), lr=0.001)

**Send model to GPU**

In [13]:
print('Is cuda available? - ', torch.cuda.is_available())

if torch.cuda.is_available():
    device = torch.device('gpu')
else:
    device = torch.device('cpu')
    
simplenet.to(device)

Is cuda available? -  False


SimpleNet(
  (fc1): Linear(in_features=12288, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
)

In [14]:
device

device(type='cpu')

**Training**

In [15]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=None, device='cpu'):
    '''
    device option copy on cpu/gpu only batch, not model. Model was sent to cpu/gpu in the cell below!
    '''
    for epoch in range(epochs):
        training_loss = 0
        val_loss = 0
        
        model.train() #set model to the train mode
        for batch in train_loader:
            optimizer.zero_grad() #set gradients to zero for every new bath
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward() # backward propagation
            optimizer.step() # update weights
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
            
        model.eval()
        num_correct = 0
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            val_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets)
            num_correct += torch.sum(correct).item()
            num_examles += correct.shape[0]
        valid_loss /= len(val_loader.dataset)
        
        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.\
              format(epoch, training_loss, valid_loss, num_correct / num_examples))

          
    

In [ ]:
train(simplenet, optimizer, nn.CrossEntropyLoss(), train_loader,val_loader, epochs=5, device=device)